# Problem description

****

Five times more deadly than the flu, COVID-19 causes significant morbidity and mortality. Like other pneumonias, pulmonary infection with COVID-19 results in inflammation and fluid in the lungs. COVID-19 looks very similar to other viral and bacterial pneumonias on chest radiographs, which makes it difficult to diagnose. This computer vision model for detection and localization of COVID-19 would help doctors provide a quick and confident diagnosis. As a result, patients could get the right treatment before the most severe effects of the virus take hold.


Currently, COVID-19 can be diagnosed via polymerase chain reaction to detect genetic material from the virus or chest radiograph. However, it can take a few hours and sometimes days before the molecular test results are back. By contrast, chest radiographs can be obtained in minutes. While guidelines exist to help radiologists differentiate COVID-19 from other types of infection, their assessments vary. In addition, non-radiologists could be supported with better localization of the disease, such as with a visual bounding box.


In this competition, the task is to identify and localize COVID-19 abnormalities on chest radiographs. In particular, categorization of the radiographs as negative for pneumonia or typical, indeterminate, or atypical for COVID-19.

**Categorization of the radiographs:**

* NEGATIVE FOR PNEUMONIA - No lung opacities

* TYPICAL APPEARANCE - Multifocal bilateral, peripheral opacities with rounded morphology, lower lung–predominant distribution

* INDETERMINATE APPEARANCE - Absence of typical findings AND unilateral, central or upper lung predominant distribution

* ATYPICAL APPEARANCE - Pneumothorax, pleural effusion, pulmonary edema, lobar consolidation, solitary lung nodule or mass, diffuse tiny nodules, cavity

**Input data:**

* train_study_level.csv - the train study-level metadata, with one row for each study, including correct labels.
* train_image_level.csv - the train image-level metadata, with one row for each image, including both correct labels and any bounding boxes in a dictionary format. Some images in both test and train have multiple bounding boxes.
* sample_submission.csv - a sample submission file containing all image- and study-level IDs.
* train folder - comprises 6334 chest scans in DICOM format, stored in paths with the form study/series/image
* test folder - The hidden test dataset is of roughly the same scale as the training dataset. Studies in the test set may contain more than one label.

# Content table

****

1. Importing the libraries
2. Importing the datasets
3. Data exploration
4. Read Dicom files
5. Feature engineering
6. Making the model
7. Compiling the model
8. References

# Importing the libraries
****

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
import pydicom as dicom # Dicom (Digital Imaging in Medicine) - medical image datasets, storage and transfer
import os
from tqdm import tqdm # allows you to output a smart progress bar by wrapping around any iterable
import glob # retrieve files/pathnames matching a specified pattern
import pprint # pretty-print” arbitrary Python data structures
import ast # 
from pydicom.pixel_data_handlers.util import apply_voi_lut #
import wandb #
import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from textwrap import wrap

pd.set_option('display.max_columns', 500)

# Importing the datasets
****

In [ ]:
path = '/kaggle/input/siim-covid19-detection/'
train_dir = '/kaggle/input/siim-covid19-detection/train'
test_dir = '/kaggle/input/siim-covid19-detection/test'

train_image_level = pd.read_csv(path + "train_image_level.csv")
train_study_level = pd.read_csv(path + "train_study_level.csv")

# Data exploration
****

Let's have a look inside the train_image_level:

In [ ]:
train_image_level.head()

In [ ]:
train_image_level.describe()

There are 6334 unique values in the train_image_level dataframe.

Now let's have a look inside train_study_level dataset:

In [ ]:
train_study_level.head()

In [ ]:
train_study_level.describe()

**Distribution of Appearances**

Note:

When you use enumerate(), the function gives you back two loop variables:

1. The count of the current iteration
2. The value of the item at the current iteration

In [ ]:
columns = ['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']
sum = []

# label rotation for clear view
fig, ax = plt.subplots()
ax.set_xticklabels(labels = columns, rotation = 45)

for column in columns:
    plt.bar(column, train_study_level[column].sum())

There are 6054 rows in the train_study_level dataframe. The number of unique values in study dataframe differs from the unique values in the images dataframe. Let's check how many studies have more than 1 image linked.

In [ ]:
train_study_level_key = train_study_level.id.str[:-6]
training_set = pd.merge(left = train_study_level, right = train_image_level, how = 'right', left_on = train_study_level_key, right_on = 'StudyInstanceUID')
training_set.drop(['id_x'], axis = 1)

Let's have a look at these studies with multiple images:

In [ ]:
training_set[training_set.groupby('StudyInstanceUID')['id_y'].transform('size') > 1].sort_values('StudyInstanceUID')

# Read Dicom files
****

Function used to locate image from the path:

In [ ]:
def extract_image(i):
    path_train = path + 'train/' + training_set.loc[i, 'StudyInstanceUID']
    last_folder_in_path = os.listdir(path_train)[0]
    path_train = path_train + '/{}/'.format(last_folder_in_path)
    img_id = training_set.loc[i, 'id_y'].replace('_image','.dcm')
    
    print(img_id)
    
    data_file = dicom.dcmread(path_train + img_id)
    img = data_file.pixel_array
    return img

**Images and rectangles visualization**

In [ ]:
fig, axes = plt.subplots(3,3, figsize=(20,16))
fig.subplots_adjust(hspace=.1, wspace=.1)
axes = axes.ravel()

for row in range(9):
    img = extract_image(row)
    if (training_set.loc[row,'boxes'] == training_set.loc[row,'boxes']):
        boxes = ast.literal_eval(training_set.loc[row,'boxes'])
        for box in boxes:
            p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                              box['width'], box['height'],
                                              ec = 'r', fc = 'none', lw = 2.
                                            )
            axes[row].add_patch(p)
    axes[row].imshow(img, cmap = 'gray')
    axes[row].set_title(training_set.loc[row, 'label'].split(' ')[0])
    axes[row].set_xticklabels([])
    axes[row].set_yticklabels([])

# Feature engineering
****

**Opacity_Count** - Count the number of opacities in the image

In [ ]:
Opacity_Count = training_set['label'].str.count('opacity')
training_set['Opacity_Count'] = Opacity_Count.values

**Rectange_Area** - Sum of areas of rectangles - assumption : the bigger the rectangle - the bigger the opacity

In [ ]:
image_rectangles_areas = []

for row in range(6334):#len(training_set.index)):
    image_rectangles_area_sum = 0
    rectangle_area = 0
    if (training_set.loc[row,'boxes'] == training_set.loc[row,'boxes']):
        boxes = ast.literal_eval(training_set.loc[row,'boxes'])
        for box in boxes:
            rectangle_area = box['width'] * box['height']
            image_rectangles_area_sum = image_rectangles_area_sum + rectangle_area
        image_rectangles_areas.append(image_rectangles_area_sum)
    else: # nan values
        image_rectangles_area_sum = image_rectangles_area_sum + rectangle_area
        image_rectangles_areas.append(image_rectangles_area_sum)

In [ ]:
training_set['Rectangle_Area'] = image_rectangles_areas

**Creating buckets - rectangle areas**

Distribution of the rectangle areas

In [ ]:
training_set['Rectangle_Area'] = round(training_set['Rectangle_Area'],2)

In [ ]:
#pd.qcut(training_set['Rectangle_Area'], q = 4)

#training_set.boxplot(by = "Negative for Pneumonia",column = ['Rectangle_Area'],grid = True, layout=(1, 1))

cut_labels_4 = ['0', '<1e6', '<2e6', '<4e6', '<8e6']
cut_bins = [-1, 0, 1000000, 2000000, 4000000, 8000000]
training_set['Rectangle_Area_Bin'] = pd.cut(training_set['Rectangle_Area'], bins = cut_bins, labels = cut_labels_4)

In [ ]:
columns = ['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']

plt.figure(figsize = (16, 14))
sn.set(font_scale = 1.2)
sn.set_style('ticks')

for i, column in enumerate(columns):
    plt.subplot(3, 3, i + 1)
    sn.countplot(data = training_set, x = 'Rectangle_Area_Bin', hue = column, palette = ['#d02f52',"#55a0ee"])
    
sn.despine()

In [ ]:
opacity = sorted(list(training_set['Rectangle_Area_Bin'].value_counts().index))

for i in opacity:
    Count_Series = training_set[training_set['Rectangle_Area_Bin'] == i].iloc[:,[1, 2, 3, 4]].sum()
    fig = plt.figure(figsize=(12,3))
    sn.barplot(x = Count_Series.index, y = Count_Series.values/sum(training_set['Rectangle_Area_Bin'] == i))
    plt.title('Rectangle_Area_Bin : {} '.format(i))
    plt.plot()

Rectangle area and opacity count

In [ ]:

opacity = sorted(list(training_set['Opacity_Count'].value_counts().index))

for i in opacity:
    Count_Series = training_set[training_set['Opacity_Count'] == i].iloc[:,[1, 2, 3, 4]].sum()
    fig = plt.figure(figsize=(12,3))
    sn.barplot(x = Count_Series.index, y = Count_Series.values/sum(training_set['Opacity_Count'] == i))
    plt.title('OpacityCount : {} '.format(i))
    plt.plot()

**TBD**: Position of the rectangle by quadrants (4 bins - 4 quadrants)

**Image metadata**

In [ ]:
training_paths = []
train_directory = "../input/siim-covid19-detection/train/"

for sid in tqdm(training_set['StudyInstanceUID']):
    training_paths.append(glob.glob(os.path.join(train_directory, sid +"/*/*"))[0])

training_set['path'] = training_paths

The pixel values are in the range of 0 to 255. It is easier for us to normalize the data between 0 to 1 and we can do that just by dividing our train and test set by 255.

In [ ]:
voi_lut=True
fix_monochrome=True

def dicom_dataset_to_dict(filename,func):
    """Credit: https://github.com/pydicom/pydicom/issues/319
               https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    """
    
    dicom_header = dicom.dcmread(filename) 
    
    #====== DICOM FILE DATA ======
    dicom_dict = {}
    repr(dicom_header)
    for dicom_value in dicom_header.values():
        if dicom_value.tag == (0x7fe0, 0x0010):
            #discard pixel data
            continue
        if type(dicom_value.value) == dicom.dataset.Dataset:
            dicom_dict[dicom_value.name] = dicom_dataset_to_dict(dicom_value.value)
        else:
            v = _convert_value(dicom_value.value)
            dicom_dict[dicom_value.name] = v
      
    del dicom_dict['Pixel Representation']
    
    if func != 'metadata_df':
        #====== DICOM IMAGE DATA ======
        # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
        if voi_lut:
            data = apply_voi_lut(dicom_header.pixel_array, dicom_header)
        else:
            data = dicom_header.pixel_array
        # depending on this value, X-ray may look inverted - fix that:
        if fix_monochrome and dicom_header.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data
        data = data - np.min(data)
        data = data / np.max(data)
        modified_image_data = (data * 255).astype(np.uint8)
    
        return dicom_dict, modified_image_data
    
    else:
        return dicom_dict

def _sanitise_unicode(s):
    return s.replace(u"\u0000", "").strip()

def _convert_value(v):
    t = type(v)
    if t in (list, int, float):
        cv = v
    elif t == str:
        cv = _sanitise_unicode(v)
    elif t == bytes:
        s = v.decode('ascii', 'replace')
        cv = _sanitise_unicode(s)
    elif t == dicom.valuerep.DSfloat:
        cv = float(v)
    elif t == dicom.valuerep.IS:
        cv = int(v)
    else:
        cv = repr(v)
    return cv



Getting the dictionary data to the dataframe and dropping the columns not needed

In [ ]:
metadata = []

for filename in training_set.path:
    try:
        data_di = dicom_dataset_to_dict(filename,'metadata_df')
        metadata.append(data_di)
    except:
        continue

dicom_data_df = pd.DataFrame(metadata)

In [ ]:
dicom_data_df.head()

In [ ]:
dicom_data_df.drop(['Specific Character Set', 'SOP Class UID','SOP Instance UID','Study Date','Study Time','Accession Number','Patient ID','Accession Number','Rows','Columns'], axis=1)

Get the metadata information as new columns in an existing dataframe

In [ ]:
training_set_merged = pd.merge(left = training_set, right = dicom_data_df, how = 'left', left_on = 'StudyInstanceUID', right_on = 'Study Instance UID')
training_set_merged.head()
training_set = training_set_merged

To be able to use y_col attribute in the ImageDataGenerator, we will create new column 'Class' containing the type of pneumonia.

In [ ]:
for row in range(6334):
    if training_set['Negative for Pneumonia'] == 1:
        pneumonia_class = 'Negative for Pneumonia'
    elif training_set['Typical Appearance'] == 1:
        pneumonia_class = 'Typical Appearance'
    elif training_set['Indeterminate Appearance'] == 1:
        pneumonia_class = 'Indeterminate Appearance'
    else:
        pneumonia_class = 'Atypical Appearance'

    training_set['Class'] = pneumonia_class

In [ ]:
training_set.head()

**TBD**: Outliers and irregularities in the data

# Building the model
****

EfficientNet is used:

The EfficientNet scaling method uniformly scales network width, depth, and resolution with a set of fixed scaling coefficients. The base EfficientNet-B0 network is based on the inverted bottleneck residual blocks of MobileNetV2, in addition to squeeze-and-excitation blocks.

**Understanding how EfficientNets works a little better:**

The core idea about Efficient Nets is the use of compound scaling - using a weighted scale of three inter-connected hyper parameters of the model - Resolution of the input, Depth of the Network and Width of the Network.


![](https://warehouse-camo.ingress.cmh1.psfhosted.org/fe998467d67d4e76b3f0c81fd7d52db053735d7c/68747470733a2f2f6c617465782e636f6465636f67732e636f6d2f706e672e6c617465783f5c696e6c696e652673706163653b5c6470697b3330307d2673706163653b5c62675f77686974652673706163653b5c626567696e7b616c69676e2a7d2673706163653b64657074683a262673706163653b642673706163653b3d2673706163653b5c616c7068612673706163653b5e2673706163653b5c7068692673706163653b5c5c2673706163653b77696474683a262673706163653b772673706163653b3d2673706163653b5c626574612673706163653b5e2673706163653b5c7068692673706163653b5c5c2673706163653b7265736f6c7574696f6e3a262673706163653b722673706163653b3d2673706163653b5c67616d6d612673706163653b5e2673706163653b5c7068692673706163653b5c656e647b616c69676e2a7d//)


When phi, the compound coefficient, is initially set to 1, we get the base configuration - in this case EfficientNetB0. We then use this configuration in a grid search to find the coefficients alpha, beta and gamma which optimize the following objective under the constraint:


![](https://warehouse-camo.ingress.cmh1.psfhosted.org/bc03bbc347eef78c683053ad5e24f5e348c5562b/68747470733a2f2f6c617465782e636f6465636f67732e636f6d2f706e672e6c617465783f5c696e6c696e652673706163653b5c6470697b3330307d2673706163653b5c626567696e7b616c69676e2a7d2673706163653b5c616c7068612673706163653b5c63646f742673706163653b5c626574612673706163653b5e2673706163653b322673706163653b5c63646f742673706163653b5c67616d6d612673706163653b5e2673706163653b322673706163653b265c617070726f782673706163653b322673706163653b5c5c2673706163653b5c616c7068612673706163653b5c67652673706163653b312c2673706163653b5c626574612673706163653b5c67652673706163653b26312c2673706163653b5c67616d6d612673706163653b5c67652673706163653b312673706163653b5c656e647b616c69676e2a7d)


Once these coefficients for alpha, beta and gamma are found, then simply scale phi, the compound coeffieints by different amounts to get a family of models with more capacity and possibly better performance.

**EfficientNet pros:**

By using shortcuts directly between the bottlenecks which connects a much fewer number of channels compared to expansion layers, combined with depthwise separable convolution which effectively **reduces computation** by almost a factor of k^2, compared to traditional layers. Where k stands for the kernel size, specifying the height and width of the 2D convolution window.

The second benefit of EfficientNet, it scales more efficiently by carefully balancing network depth, width, and resolution, which lead to **better performance**.

****
Next we set up the infrastructure to run a training job on our dataset. We choose the number of epochs to train for. The more epochs, the better your model is likely to fit your data but training will run for longer.

Next, we set up the network to build the correct number of layers for the number of classes we have in our dataset.

**Layers:**


**The pooling layer** operates upon each feature map separately to create a new set of the same number of pooled feature maps.
Pooling involves selecting a pooling operation, much like a filter to be applied to feature maps.
Average Pooling: Calculate the average value for each patch on the feature map.

**Dense** implements the operation: output = activation(dot(input, kernel) + bias) where activation is the element-wise activation function passed as the activation argument, kernel is a weights matrix created by the layer, and bias is a bias vector created by the layer (only applicable if use_bias is True).
Dense is the only actual network layer in the model.
A Dense layer feeds all outputs from the previous layer to all its neurons, each neuron providing one output to the next layer. A Dense(10) has ten neurons.

**Model** groups layers into an object with training and inference features.

**Adam** is a replacement optimization algorithm for stochastic gradient descent for training deep learning models. Adam combines the best properties of the AdaGrad and RMSProp algorithms to provide an optimization algorithm that can handle sparse gradients on noisy problems.

Loss is a prediction error of Neural Net. And the method to calculate the loss is called Loss Function. In simple words, the Loss is used to calculate the gradients. And gradients are used to update the weights of the Neural Net.

**CategoricalCrossentropy** - crossentropy loss function when there are two or more label classes

****

In [ ]:
EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]

def build_model(dim = IMG_SIZES[0], ef = 0):
    inputs = tf.keras.layers.Input(shape = (*dim, 3))
    base = EFNS[ef](input_shape = (*dim,3), weights = 'imagenet', include_top = False)
    x = base(inputs)
    
    # pooling layer
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    # Dense
    x = tf.keras.layers.Dense(64, activation = 'relu')(x)
    x = tf.keras.layers.Dense(4, activation = 'softmax')(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = x)
    
    opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
    
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.01)
    
    auc = tf.keras.metrics.AUC(curve = 'ROC', multi_label = True)
    
    acc = tf.keras.metrics.CategoricalAccuracy()
    
    f1  = tfa.metrics.F1Score(num_classes = 4,average = 'macro',threshold = None)
    
    model.compile(optimizer = opt,loss = loss,metrics = [auc, acc, f1])
    
    return model

# Training the model
****

1. Specify where the training and test folders are
2. Use Keras's **ImageDataGenerator** to **augment** the training data. If you haven't used this library before, or are new to data augmentation, take a look at this link: http://keras.io/preprocessing/image/
3. Use a pre-trained model called **EfficientNet**
4. Make predictions on the test images in the test zip file and format the submission.csv file to hold our own submissions!

Image data augmentation is a technique that can be used to artificially expand the size of a training dataset by creating modified versions of images in the dataset.

**Augmenting the images: ImageDataGenerator:**

* generate **two generators** - one for training, and another for validation. These are stored in train_generator and val_generator. For both, we apply a series of distortions.
* instead of storing all these new images in a directory, we use the method flow_from_dataframe to dynamically load these images as we train the model
* **flow_from_dataframe** - takes the dataframe and the path to a directory + generates batches. The generated batches contain augmented/normalized data.
* all the distortions we made for **train_gen** are not applied to test_gen. This is because we don't want to augment the data in the test directory.

* **training** dataset - used to fit the model
* **validation** dataset - used to provide an unbiased evaluation of a model fit on the training dataset while tuning model hyperparameters

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    rescale = 1/255,
    validation_split = 0.10,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_dg = data_generator.flow_from_dataframe(
dataframe = training_set,
directory = ,
x_col = ,
y_col = ,
target_size = (),
subset = ,
batch_size = 1024,
shuffle = True,
class_mode = 'categorical')

In [ ]:
folds = 5
epochs = [12] * folds

# train and validation subsets

num_of_train_files = len(train_image_level)

print(num_of_train_files)

**KFold cross validation**
****
Provides train/test indices to split data in train/test sets. Split dataset into k consecutive folds (without shuffling by default).

Each fold is then used once as a validation while the k - 1 remaining folds form the training set.

When shuffle is True, random_state affects the ordering of the indices, which controls the randomness of each fold. Otherwise, this parameter has no effect. 

split(X[, y, groups]) - Generate indices to split data into training and test set.

In [ ]:
from sklearn.model_selection import KFold

skf = KFold(n_splits = folds, shuffle = True, random_state = 0)


In [ ]:
np.arange(num_of_train_files)
skf.split(np.arange(num_of_train_files))

# References
****

* https://github.com/pydicom/pydicom/issues/319
* https://www.kaggle.com/songseungwon/siim-covid-19-detection-10-step-tutorial-1
* https://www.kaggle.com/ruchi798/siim-covid-19-detection-eda-data-augmentation#DICOM-data
* https://www.kaggle.com/awsaf49/siim-covid-19-study-level-train-tpu/comments
* https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7
* https://www.kaggle.com/arjunrao2000/beginners-guide-efficientnet-with-keras